## Settings

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

# Utilities
import warnings, time
from tqdm import tqdm
import time

# Allows to get weather data
from meteostat import Stations
from meteostat import Daily

# Options
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

In [2]:
data_path = 'data/world_disasters.xlsx'

gdp_url = 'https://pkgstore.datahub.io/core/gdp/gdp_csv/data/0048bc8f6228d0393d41cac4b663b90f/gdp_csv.csv'
pop_url = 'https://datahub.io/core/population/r/population.csv'

pop_values_url = 'https://www.macrotrends.net/countries/ranking/population'
gdp_values_url = 'https://www.macrotrends.net/countries/ranking/gdp-per-capita'
sub_regions_url = 'https://raw.githubusercontent.com/lukes/ISO-3166-Countries-with-Regional-Codes/master/all/all.csv'

## Data Read and Exploration

In [3]:
data = pd.read_excel(data_path)
print(f'Data Size: {data.shape[0]} x {data.shape[1]}')
data.head()

Data Size: 18800 x 50


,Dis No,Year,Seq,Glide,Disaster Group,Disaster Subgroup,Disaster Type,Disaster Subtype,Disaster Subsubtype,Event Name,Country,ISO,Region,Continent,Location,Origin,Associated Dis,Associated Dis2,OFDA Response,Appeal,Declaration,AID Contribution ('000 US$),Dis Mag Value,Dis Mag Scale,Latitude,Longitude,Local Time,River Basin,Start Year,Start Month,Start Day,End Year,End Month,End Day,Total Deaths,No Injured,No Affected,No Homeless,Total Affected,Reconstruction Costs ('000 US$),"Reconstruction Costs, Adjusted ('000 US$)",Insured Damages ('000 US$),"Insured Damages, Adjusted ('000 US$)",Total Damages ('000 US$),"Total Damages, Adjusted ('000 US$)",CPI,Adm Level,Admin1 Code,Admin2 Code,Geo Locations
0,1993-0011-COL,1993,11,NaN,Natural,Hydrological,Flood,Flash flood,NaN,NaN,Colombia,COL,South America,Americas,"Andes (Antioquia Department), Amazonas, Meta, ...",Brief torrential rain,"Slide (land, mud, snow, rock)",NaN,NaN,No,No,200.0,20.0,Km2,NaN,NaN,NaN,Taparto,1993,4.0,26.0,1993,5.0,1.0,63.0,33.0,80000.0,250.0,80283.0,NaN,NaN,NaN,NaN,NaN,NaN,49.361332,NaN,NaN,NaN,NaN
1,1993-0065-CHE,1993,65,NaN,Natural,Meteorological,Storm,Convective storm,Hail,NaN,Switzerland,CHE,Western Europe,Europe,NaN,NaN,Hail,NaN,NaN,No,No,NaN,NaN,Kph,NaN,NaN,NaN,NaN,1993,7.0,4.0,1993,7.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15000.0,30388.0,22000.0,44569.0,49.361332,NaN,NaN,NaN,NaN
2,1993-0065-DEU,1993,65,NaN,Natural,Meteorological,Storm,Convective storm,Hail,NaN,Germany,DEU,Western Europe,Europe,NaN,NaN,Hail,NaN,NaN,No,No,NaN,NaN,Kph,NaN,NaN,NaN,NaN,1993,7.0,5.0,1993,7.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25000.0,50647.0,30000.0,60776.0,49.361332,NaN,NaN,NaN,NaN
3,1993-0211-CHE,1993,211,NaN,Natural,Meteorological,Storm,NaN,NaN,NaN,Switzerland,CHE,Western Europe,Europe,NaN,NaN,Flood,NaN,NaN,No,No,NaN,NaN,Kph,NaN,NaN,NaN,NaN,1993,9.0,22.0,1993,9.0,25.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,240000.0,486211.0,420000.0,850868.0,49.361332,NaN,NaN,NaN,NaN
4,1993-0419-AFG,1993,419,NaN,Natural,Hydrological,Landslide,Avalanche,NaN,NaN,Afghanistan,AFG,Southern Asia,Asia,Salang district (Parwan province),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1993,3.0,NaN,1993,3.0,NaN,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,49.361332,NaN,NaN,NaN,NaN


### Missing Values

In [4]:
missing = data.isnull().sum().reset_index()
missing.columns = ['Column', 'Count_of_NaN']
missing['Total'] = len(data)
missing['Ratio_of_NaN_%'] =  np.round(100 * missing['Count_of_NaN'] / missing['Total'], 2)
missing.drop(columns = ['Total'], inplace = True)
missing.sort_values(by = ['Ratio_of_NaN_%'], ascending = False, inplace = True)
missing.set_index('Column')

,Count_of_NaN,Ratio_of_NaN_%
Column,,
Reconstruction Costs ('000 US$),18764,99.81
"Reconstruction Costs, Adjusted ('000 US$)",18765,99.81
Local Time,18207,96.85
Associated Dis2,18092,96.23
AID Contribution ('000 US$),18039,95.95
Disaster Subsubtype,17909,95.26
"Insured Damages, Adjusted ('000 US$)",17901,95.22
Insured Damages ('000 US$),17901,95.22
OFDA Response,17884,95.13


### Unique values & Data Types

In [5]:
data.nunique()

Dis No                                       18800
Year                                            31
Seq                                           1260
Glide                                         1232
Disaster Group                                   3
Disaster Subgroup                                8
Disaster Type                                   18
Disaster Subtype                                41
Disaster Subsubtype                             10
Event Name                                    2562
Country                                        224
ISO                                            224
Region                                          22
Continent                                        5
Location                                     15753
Origin                                         792
Associated Dis                                  32
Associated Dis2                                 29
OFDA Response                                    1
Appeal                         

In [6]:
data[data['Start Month'].notnull() & data['End Month'].notnull() & (data['Disaster Type'] == 'Epidemic')]

,Dis No,Year,Seq,Glide,Disaster Group,Disaster Subgroup,Disaster Type,Disaster Subtype,Disaster Subsubtype,Event Name,Country,ISO,Region,Continent,Location,Origin,Associated Dis,Associated Dis2,OFDA Response,Appeal,Declaration,AID Contribution ('000 US$),Dis Mag Value,Dis Mag Scale,Latitude,Longitude,Local Time,River Basin,Start Year,Start Month,Start Day,End Year,End Month,End Day,Total Deaths,No Injured,No Affected,No Homeless,Total Affected,Reconstruction Costs ('000 US$),"Reconstruction Costs, Adjusted ('000 US$)",Insured Damages ('000 US$),"Insured Damages, Adjusted ('000 US$)",Total Damages ('000 US$),"Total Damages, Adjusted ('000 US$)",CPI,Adm Level,Admin1 Code,Admin2 Code,Geo Locations
33,1993-0046-BGD,1993,46,NaN,Natural,Biological,Epidemic,NaN,NaN,NaN,Bangladesh,BGD,Southern Asia,Asia,"Banderban, Sylhet, Brahmanbaria, Hobiganj, Mou...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Vaccinated,NaN,NaN,NaN,NaN,1993,8.0,NaN,1993,8.0,NaN,38.0,NaN,5660.0,NaN,5660.0,NaN,NaN,NaN,NaN,NaN,NaN,49.361332,NaN,NaN,NaN,NaN
101,1993-0500-CMR,1993,500,NaN,Natural,Biological,Epidemic,Bacterial disease,NaN,NaN,Cameroon,CMR,Middle Africa,Africa,Far North and North provinces,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Vaccinated,NaN,NaN,NaN,NaN,1993,2.0,NaN,1993,2.0,NaN,513.0,NaN,4070.0,NaN,4070.0,NaN,NaN,NaN,NaN,NaN,NaN,49.361332,NaN,NaN,NaN,NaN
474,1993-0562-USA,1993,562,NaN,Natural,Biological,Epidemic,Parasitic disease,NaN,Cryptosporidiosis,United States of America (the),USA,Northern America,Americas,Wisconsin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Vaccinated,NaN,NaN,NaN,NaN,1993,1.0,NaN,1993,1.0,NaN,100.0,NaN,403000.0,NaN,403000.0,NaN,NaN,NaN,NaN,NaN,NaN,49.361332,NaN,NaN,NaN,NaN
517,1994-0018-SOM,1994,18,NaN,Natural,Biological,Epidemic,NaN,NaN,NaN,Somalia,SOM,Eastern Africa,Africa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Vaccinated,NaN,NaN,NaN,NaN,1994,3.0,NaN,1994,3.0,NaN,100.0,NaN,17000.0,NaN,17000.0,NaN,NaN,NaN,NaN,NaN,NaN,50.648403,NaN,NaN,NaN,NaN
655,1995-0407-BGD,1995,407,NaN,Natural,Biological,Epidemic,Parasitic disease,NaN,NaN,Bangladesh,BGD,Southern Asia,Asia,"Sunamganj, Chittagong and Cox's Bazar",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Vaccinated,NaN,NaN,NaN,NaN,1995,3.0,NaN,1995,3.0,NaN,350.0,NaN,20000.0,NaN,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,52.069299,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18710,2022-0201-TLS,2022,201,EP-2022-000162,Natural,Biological,Epidemic,Viral disease,NaN,Dengue,Timor-Leste,TLS,South-Eastern Asia,Asia,"Aileu, Ainaro, Baucau, Bobonaro, Covalima, Dil...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Vaccinated,NaN,NaN,NaN,NaN,2022,1.0,NaN,2022,1.0,31.0,20.0,NaN,1353.0,NaN,1353.0,NaN,NaN,NaN,NaN,NaN,NaN,100.000000,NaN,NaN,NaN,NaN
18747,2023-0166-NGA,2023,166,EP-2023-000034,Natural,Biological,Epidemic,Bacterial disease,NaN,Diphteria,Nigeria,NGA,Western Africa,Africa,"Kano, Yobe, Katsina, Sokoto, Enugu, Ogun, Osun...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Vaccinated,NaN,NaN,NaN,NaN,2023,1.0,20.0,2023,3.0,23.0,89.0,733.0,NaN,NaN,733.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2224;2225;2226;2230;2232;2234;2240;65703;65709...,NaN,"Enugu, Kaduna, Kano, Katsina, Lagos, Ogun, Osu..."
18769,2023-0201-SOM,2023,201,NaN,Natural,Biological,Epidemic,Bacterial disease,NaN,Acute watery Diarrhoea/cholera,Somalia,SOM,Eastern Africa,Africa,"Kismayo, Belethawo, Afgoye, Afmadow",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Vaccinated,NaN,NaN,NaN,NaN,2023,1.0,1.0,2023,4.0,4.0,14.0,3428.0,NaN,NaN,3428.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,25679;25687;25690;25691,"Afgooye, Afmadow, Belet Xaawo, Kismaayo (Adm2)."
18796,2023-0068-ZMB,2023,68,EP-2023-000013,Natural,Biological,Epidemic,Bacterial disease,NaN,Cholera,Zambia,ZMB,Eastern Africa,Africa,"Chikoma, Mbande, Mzigawa, Chipanji, Mlawe, Eas...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Vaccinated,NaN,NaN,NaN,NaN,2023,1.0,21.0,2023,1.0,30.0,2.0,118.0,NaN,NaN,118.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,3428;

## Basic preprocessing

- Remove useless features
- Fix Missing Dates
- Transform Features

**NOTE:** for each disaster type and region let's compute average period length to fill missing ones

In [7]:
drop_list = [
    'Admin1 Code',
    'Admin2 Code',
    'Disaster Subsubtype',
    'Glide',
    'Event Name',
#     'Country',
    'Location',
    'Latitude',
    'Longitude',
    'River Basin',
    'CPI',
    'Adm Level',
    'Geo Locations',
    'Associated Dis2',
    'OFDA Response',
    'Appeal',
    'Declaration',
    'Local Time',
    'Year'
]

date_cols = ['Start Day', 'Start Month', 'Start Year', 'End Day', 'End Month', 'End Year']
data.drop(columns = drop_list, inplace = True)
data['Disaster Subtype'].fillna(data['Disaster Type'], inplace = True)

In [8]:
def date_creator(df, year_col, month_col, day_col):
    def zero_adder(value):
        if len(value) == 1:
            value = '0' + value
        return value
    
    df[month_col] = df[month_col].apply(zero_adder)
    df[day_col] = df[day_col].apply(zero_adder)
    
    dates = pd.to_datetime((df[year_col] + '-' + df[month_col] + '-' + df[day_col])).values
    return dates

non_missing_dates = data[data['Start Month'].notnull() & data['End Month'].notnull()].copy()
non_missing_dates['Start Day'].fillna(1, inplace = True)
non_missing_dates['End Day'].fillna(1, inplace = True)
non_missing_dates['Disaster Subtype'].fillna(non_missing_dates['Disaster Type'], inplace = True)

for col in date_cols:
    non_missing_dates[col] = non_missing_dates[col].astype(int).astype(str)
    
non_missing_dates['Start_Date'] = date_creator(df=non_missing_dates, 
                                               year_col='Start Year',
                                               month_col='Start Month',
                                               day_col='Start Day')

non_missing_dates['End_Date'] = date_creator(df=non_missing_dates, 
                                               year_col='End Year',
                                               month_col='End Month',
                                               day_col='End Day')

non_missing_dates['Duration'] = (non_missing_dates['End_Date'] - non_missing_dates['Start_Date']).dt.days
non_missing_dates = non_missing_dates.drop(columns = date_cols)

# Find mean duration for each disaster type and Country
mean_duration_1 = non_missing_dates[['ISO', 'Duration', 'Disaster Subtype']].groupby(by = ['ISO', 'Disaster Subtype']).mean()
mean_duration_1['Duration'] = np.round(mean_duration_1['Duration'].apply(lambda x: 1 if x < 1 else x), 2)
mean_duration_1.reset_index(inplace = True)
print('Mean Disaster duration for each Country')
display(mean_duration_1.head())

# Find mean duration for each disaster type and Region
mean_duration_2 = non_missing_dates[['Region', 'Duration', 'Disaster Subtype']].groupby(by = ['Region', 'Disaster Subtype']).mean()
mean_duration_2['Duration'] = np.round(mean_duration_2['Duration'].apply(lambda x: 1 if x < 1 else x), 2)
mean_duration_2.reset_index(inplace = True)
print('\nMean Disaster duration for each Region')
display(mean_duration_2.head())

# Find mean duration for each disaster type and Continent
mean_duration_3 = non_missing_dates[['Continent', 'Duration', 'Disaster Subtype']].groupby(by = ['Continent', 'Disaster Subtype']).mean()
mean_duration_3['Duration'] = np.round(mean_duration_3['Duration'].apply(lambda x: 1 if x < 1 else x), 2)
mean_duration_3.reset_index(inplace = True)
print('\nMean Disaster duration for each Continent')
display(mean_duration_3.head())

# Find mean duration for each disaster
mean_duration_4 = non_missing_dates[['Duration', 'Disaster Subtype']].groupby(by = ['Disaster Subtype']).mean()
mean_duration_4['Duration'] = np.round(mean_duration_4['Duration'].apply(lambda x: 1 if x < 1 else x), 2)
mean_duration_4.reset_index(inplace = True)
print('\nMean Disaster duration for each Disaster type')
display(mean_duration_4.head())

Mean Disaster duration for each Country


,ISO,Disaster Subtype,Duration
0,AFG,Air,1.00
1,AFG,Avalanche,1.00
2,AFG,Bacterial disease,72.17
3,AFG,Cold wave,8.75
4,AFG,Collapse,1.00



Mean Disaster duration for each Region


,Region,Disaster Subtype,Duration
0,Australia and New Zealand,Air,1.00
1,Australia and New Zealand,Ash fall,1.00
2,Australia and New Zealand,Coastal flood,13.00
3,Australia and New Zealand,Convective storm,1.68
4,Australia and New Zealand,Drought,182.00



Mean Disaster duration for each Continent


,Continent,Disaster Subtype,Duration
0,Africa,Air,1.00
1,Africa,Animal accident,1.00
2,Africa,Ash fall,8.67
3,Africa,Bacterial disease,82.00
4,Africa,Chemical spill,1.00



Mean Disaster duration for each Disaster type


,Disaster Subtype,Duration
0,Air,1.00
1,Animal accident,1.00
2,Ash fall,10.23
3,Avalanche,1.96
4,Bacterial disease,75.20


Now let's compute empty dates, with mean time for each Disaster type in each Geo-location 

In [9]:
# We know month of end, but no month of start
part1 = data[data['Start Month'].isnull() & data['End Month'].notnull()].copy()

part1['End Day'] = part1['End Day'].fillna(15)
for col in date_cols[-3:]:
    part1[col] = part1[col].astype(int).astype(str)

part1['End_Date'] = date_creator(df=part1, 
                                 year_col='End Year',
                                 month_col='End Month',
                                 day_col='End Day')

part1['Start_Date'] = np.nan

for frame in [mean_duration_1, mean_duration_2, mean_duration_2, mean_duration_3]:
    part1 = part1.merge(frame, how = 'left', on = list(frame.drop(columns = ['Duration']).columns))
    part1['Start_Date'].fillna(part1['End_Date'] - pd.to_timedelta(part1['Duration'], unit='day'), inplace = True)
    part1 = part1.drop(columns = ['Duration'])
    nan_ratio = np.round(100 * len(part1[part1['Start_Date'].isnull()]) / len(part1), 1)
    if nan_ratio == 0:
        break

part1['Start_Date'] = part1['Start_Date'].dt.date
part1['End_Date'] = part1['End_Date'].dt.date
part1 = part1.drop(columns = date_cols)

#########################################################################

# We know month of start, but no month of end
part2 = data[data['Start Month'].notnull() & data['End Month'].isnull()].copy()

part2['Start Day'] = part2['Start Day'].fillna(15)
for col in date_cols[:-3]:
    part2[col] = part2[col].astype(int).astype(str)

part2['Start_Date'] = date_creator(df=part2, 
                                 year_col='Start Year',
                                 month_col='Start Month',
                                 day_col='Start Day')

part2['End_Date'] = np.nan

for frame in [mean_duration_1, mean_duration_2, mean_duration_2, mean_duration_3]:
    part2 = part2.merge(frame, how = 'left', on = list(frame.drop(columns = ['Duration']).columns))
    part2['End_Date'].fillna(part2['Start_Date'] + pd.to_timedelta(part2['Duration'], unit='day'), inplace = True)
    part2 = part2.drop(columns = ['Duration'])
    nan_ratio = np.round(100 * len(part2[part2['End_Date'].isnull()]) / len(part2), 1)
    if nan_ratio == 0:
        break

part2['Start_Date'] = part2['Start_Date'].dt.date
part2['End_Date'] = part2['End_Date'].dt.date
part2 = part2.drop(columns = date_cols)


#########################################################################

# We know both month of start, and month of end
part3 = data[data['Start Month'].notnull() & data['End Month'].notnull()].copy()
part3['Start Day'].fillna(15, inplace = True)
part3['End Day'].fillna(15, inplace = True)

for col in date_cols:
    part3[col] = part3[col].astype(int).astype(str)

part3['Start_Date'] = date_creator(df=part3, 
                                   year_col='Start Year',
                                   month_col='Start Month',
                                   day_col='Start Day')

part3['End_Date'] = date_creator(df=part3, 
                                 year_col='End Year',
                                 month_col='End Month',
                                 day_col='End Day')

part3['Start_Date'] = part3['Start_Date'].dt.date
part3['End_Date'] = part3['End_Date'].dt.date
part3 = part3.drop(columns = date_cols)


#########################################################################
# Merge together

df = pd.concat([part1, part2, part3], axis=0, ignore_index=True)
df['Start_Date'].fillna(df['End_Date'], inplace = True)
df['End_Date'].fillna(df['Start_Date'], inplace = True)
df['Duration'] = (df['End_Date'] - df['Start_Date']).dt.days
df['Duration'] = np.round(df['Duration'].apply(lambda x: 1 if x < 1 else x), 2)
df['Duration'] = df['Duration'].astype(int)
df['Year'] = pd.to_datetime(df['Start_Date']).dt.year
df['Month'] = pd.to_datetime(df['Start_Date']).dt.month
df.drop_duplicates(subset = ['Dis No'], keep = 'last', inplace = True)
print(f'Data Size: {df.shape[0]} x {df.shape[1]}')
df.head()


Data Size: 18748 x 31


,Dis No,Seq,Disaster Group,Disaster Subgroup,Disaster Type,Disaster Subtype,Country,ISO,Region,Continent,Origin,Associated Dis,AID Contribution ('000 US$),Dis Mag Value,Dis Mag Scale,Total Deaths,No Injured,No Affected,No Homeless,Total Affected,Reconstruction Costs ('000 US$),"Reconstruction Costs, Adjusted ('000 US$)",Insured Damages ('000 US$),"Insured Damages, Adjusted ('000 US$)",Total Damages ('000 US$),"Total Damages, Adjusted ('000 US$)",End_Date,Start_Date,Duration,Year,Month
0,1998-0570-BOL,570,Natural,Biological,Epidemic,Bacterial disease,Bolivia (Plurinational State of),BOL,South America,Americas,NaN,NaN,NaN,NaN,Vaccinated,5.0,NaN,165.0,NaN,165.0,NaN,NaN,NaN,NaN,NaN,NaN,1998-03-31,1998-03-16,15,1998,3
1,1998-0570-HND,570,Natural,Biological,Epidemic,Bacterial disease,Honduras,HND,Central America,Americas,NaN,NaN,NaN,NaN,Vaccinated,17.0,NaN,1372.0,NaN,1372.0,NaN,NaN,NaN,NaN,NaN,NaN,1998-12-15,1998-12-02,13,1998,12
2,1998-0596-ZWE,596,Natural,Biological,Epidemic,Bacterial disease,Zimbabwe,ZWE,Eastern Africa,Africa,NaN,NaN,NaN,NaN,Vaccinated,12.0,NaN,335.0,NaN,335.0,NaN,NaN,NaN,NaN,NaN,NaN,1998-03-27,1998-01-22,64,1998,1
3,1999-0719-ZMB,719,Natural,Biological,Epidemic,Epidemic,Zambia,ZMB,Eastern Africa,Africa,NaN,NaN,NaN,NaN,Vaccinated,393.0,NaN,11327.0,NaN,11327.0,NaN,NaN,NaN,NaN,NaN,NaN,1999-06-15,1999-06-14,1,1999,6
4,2006-0727-PRY,727,Natural,Biological,Epidemic,Viral disease,Paraguay,PRY,South America,Americas,NaN,NaN,NaN,NaN,Vaccinated,17.0,NaN,100000.0,NaN,100000.0,NaN,NaN,NaN,NaN,NaN,NaN,2007-04-15,2006-11-22,144,2006,11


## Add external data
- GDP
- Population
- Weather

In [10]:
replacements = {
                        'Russian Federation' : 'Russia',
                        'United States of America' : 'United States',
                        'United Kingdom of Great Britain and Northern Ireland' : 'United Kingdom',
                        "Democratic People's Republic of Korea" : 'South Korea',
                        'China, Hong Kong Special Administrative Region' : 'Hong Kong',
                        'China, Macao Special Administrative Region' : 'Macao',
                        'United Republic of Tanzania' : 'Tanzania',
                        'United Arab Emirates' : 'UAE',
                        'Slovak Republic': 'Slovakia',
                        'Republic of Moldova' : 'Moldova',
                        'Kyrgyz Republic' : 'Kyrgyzstan',
                        'Czech Republic' : 'Czechia',
                        'Iran (Islamic Republic of)' : 'Iran',
                        'Bosnia and Herzegovina' : 'Bosnia',
                        'Bolivia (Plurinational State of)' : 'Bolivia'
                    }



def get_gdp():
    gdp_data = pd.read_csv(gdp_url)
    pop_data = pd.read_csv(pop_url)

    gdp_data.columns = ['Country', 'ISO', 'Year', 'GDP']
    pop_data.columns = ['Country', 'ISO', 'Year', 'Population']
    pop_data = pop_data.drop(columns = ['Country'])
    gdp_per_caita = gdp_data.merge(pop_data, how = 'left', on = ['ISO', 'Year'])
    gdp_per_caita['GDP_per_capita'] = np.round(gdp_per_caita['GDP'] / gdp_per_caita['Population'], 2)
    gdp_per_caita = gdp_per_caita.drop(columns = ['Country'])
    print(f'Data Size: {gdp_per_caita.shape[0]} x {gdp_per_caita.shape[1]}')
    return gdp_per_caita

def get_additional_pop():
    iso_codes = pd.read_csv(sub_regions_url)
    iso_codes = iso_codes[['name', 'alpha-3', 'sub-region']]
    iso_codes = iso_codes.rename(
                        columns = {
                            "alpha-3": "Nationality",
                            'name' : 'Country Name'})
    iso_codes = iso_codes.replace({'Country Name' : replacements})
    raw_data = pd.read_html(pop_values_url)
    population= raw_data[0]
    pop = iso_codes.merge(population, how = 'left', on = ['Country Name'])
    pop.dropna(inplace = True)
    frames = []
    for col in ['2021', '2020', '2019', '2018', '2017']:
        tmp = pop[['Country Name', 'Nationality', col]]
        tmp.rename(columns = {col: 'Population'}, inplace = True)
        tmp['Year'] = col
        frames.append(tmp)

    pop_after_2016 = pd.concat(frames)
    pop_after_2016['Population'] = pop_after_2016['Population'].astype(int)
    pop_after_2016.rename(columns = {'Nationality' : 'ISO'}, inplace = True)
    return pop_after_2016

def get_additional_gdp():
    iso_codes = pd.read_csv(sub_regions_url)
    iso_codes = iso_codes[['name', 'alpha-3', 'sub-region']]
    iso_codes = iso_codes.rename(
                        columns = {
                            "alpha-3": "Nationality",
                            'name' : 'Country Name'})
    iso_codes = iso_codes.replace({'Country Name' : replacements})
    raw_data = pd.read_html(gdp_values_url)
    gdp_per_capita = raw_data[0]
    gdps = iso_codes.merge(gdp_per_capita, how = 'left', on = ['Country Name'])
    gdps.dropna(inplace = True)
    frames = []
    for col in ['2021', '2020', '2019', '2018', '2017']:
        tmp = gdps[['Country Name', 'Nationality', col]]
        tmp.rename(columns = {col: 'GDP_per_capita'}, inplace = True)
        tmp['Year'] = col
        frames.append(tmp)

    gdps_after_2016 = pd.concat(frames)
    gdps_after_2016['GDP_per_capita'] = gdps_after_2016['GDP_per_capita'].str.replace('$', '').str.replace(',', '')
    gdps_after_2016['GDP_per_capita'] = gdps_after_2016['GDP_per_capita'].astype(float)
    gdps_after_2016.rename(columns = {'Nationality' : 'ISO'}, inplace = True)
    
    population = get_additional_pop()
    population = population[['ISO', 'Year', 'Population']]
    gdps_after_2016 = gdps_after_2016.merge(population, how = 'left', on = ['ISO', 'Year'])
    gdps_after_2016['GDP'] = gdps_after_2016['Population'] * gdps_after_2016['GDP_per_capita']
    gdps_after_2016 = gdps_after_2016[['ISO', 'Year', 'GDP', 'Population', 'GDP_per_capita']]
    return gdps_after_2016




In [11]:
# Obtains GDP & Population Data
gdps = pd.concat([get_gdp(), get_additional_gdp()], axis = 0)
gdps.sort_values(by = ['ISO', 'Year'], ignore_index = True, inplace = True)
gdps

Data Size: 11507 x 5


,ISO,Year,GDP,Population,GDP_per_capita
0,ABW,1994,1.330168e+09,76700.0,17342.47
1,ABW,1995,1.320670e+09,80324.0,16441.79
2,ABW,1996,1.379888e+09,83200.0,16585.20
3,ABW,1997,1.531844e+09,85451.0,17926.57
4,ABW,1998,1.665363e+09,87277.0,19081.35
...,...,...,...,...,...
12422,ZWE,2017,1.758331e+10,14751101.0,1192.00
12423,ZWE,2018,3.415341e+10,15052184.0,2269.00
12424,ZWE,2019,2.183425e+10,15354608.0,1422.00
12425,ZWE,2020,2.151445e+10,15669666.0,1373.00


In [12]:
def get_weather_stations():
    def find_station(alpha_2_code):
        stations = Stations()
        stations = stations.region(alpha_2_code)
        w_stations = stations._data
        min_d = w_stations[['hourly_start','hourly_end','daily_start','daily_end','monthly_start','monthly_end']].min(axis = 1)
        max_d = w_stations[['hourly_start','hourly_end','daily_start','daily_end','monthly_start','monthly_end']].max(axis = 1)
        lifespans = pd.concat([min_d, max_d], axis = 1)
        lifespans['Duration'] = (pd.to_datetime(lifespans[1]) - pd.to_datetime(lifespans[0])).dt.days
        station_id = lifespans[lifespans['Duration'] == lifespans['Duration'].max()].index[0]
        return station_id

    regions = pd.read_csv(sub_regions_url)
    regions = regions[['name', 'alpha-2', 'alpha-3']]
    wstations = []
    for alpha_2_code in tqdm(regions['alpha-2'].to_list()):
        try:
            wstations.append(find_station(alpha_2_code))
            time.sleep(2) # time limit for safe API use
#             print(alpha_2_code, '-- OK')
        except:
#             print(alpha_2_code, '-- Fail')
            wstations.append(np.nan)
            continue
    regions['weather_station'] = wstations
    return regions

def get_weather(regions):
    regions = regions.dropna(subset = ['weather_station'])
    codes = regions['alpha-3'].to_list()
    wss = regions['weather_station'].to_list()
    frames = []
    for i, ws in tqdm(enumerate(wss)):
        try:
            data = Daily(ws,  datetime(1970, 1, 1), datetime(2022, 1, 1))
            frame = data.fetch()
            frame = frame[['tmin', 'tmax', 'prcp']]
            frame['ISO'] = codes[i]
            frame = frame.dropna(subset = ['tmax'])
            frames.append(frame.reset_index())
            time.sleep(2)
        except:
            continue
            
    return pd.concat(frames, axis = 0, ignore_index = True)
            

In [13]:
# May Take a while (~7mins)
regions = get_weather_stations()
regions.to_csv('ws.csv', index = False)

In [14]:
regions = pd.read_csv('ws.csv')
weather_df = get_weather(regions)
weather_df.to_csv('weather.csv', index = False)
weather_df = pd.read_csv('weather.csv')
print(f'Data Size: {weather_df.shape[0]} x {weather_df.shape[1]}')
weather_df.head()

Data Size: 2254175 x 5


,time,tmin,tmax,prcp,ISO
0,2002-11-22,-1.3,16.1,0.0,AFG
1,1973-04-30,11.0,23.0,0.5,ALB
2,1973-06-28,15.0,30.0,0.0,ALB
3,1973-08-25,18.0,32.0,0.0,ALB
4,1974-05-31,NaN,27.0,NaN,ALB


In [15]:
weather_df.rename(columns = {'time' : 'Start_Date'}, inplace = True)
weather_df['Start_Date'] = pd.to_datetime(weather_df['Start_Date']).dt.date
df = df.merge(gdps, how = 'left', on = ['ISO', 'Year'])
df = df.merge(weather_df, how = 'left', on = ['ISO', 'Start_Date'])
df.head()

,Dis No,Seq,Disaster Group,Disaster Subgroup,Disaster Type,Disaster Subtype,Country,ISO,Region,Continent,Origin,Associated Dis,AID Contribution ('000 US$),Dis Mag Value,Dis Mag Scale,Total Deaths,No Injured,No Affected,No Homeless,Total Affected,Reconstruction Costs ('000 US$),"Reconstruction Costs, Adjusted ('000 US$)",Insured Damages ('000 US$),"Insured Damages, Adjusted ('000 US$)",Total Damages ('000 US$),"Total Damages, Adjusted ('000 US$)",End_Date,Start_Date,Duration,Year,Month,GDP,Population,GDP_per_capita,tmin,tmax,prcp
0,1998-0570-BOL,570,Natural,Biological,Epidemic,Bacterial disease,Bolivia (Plurinational State of),BOL,South America,Americas,NaN,NaN,NaN,NaN,Vaccinated,5.0,NaN,165.0,NaN,165.0,NaN,NaN,NaN,NaN,NaN,NaN,1998-03-31,1998-03-16,15,1998,3,8.497546e+09,8096761.0,1049.50,3.8,13.4,7.9
1,1998-0570-HND,570,Natural,Biological,Epidemic,Bacterial disease,Honduras,HND,Central America,Americas,NaN,NaN,NaN,NaN,Vaccinated,17.0,NaN,1372.0,NaN,1372.0,NaN,NaN,NaN,NaN,NaN,NaN,1998-12-15,1998-12-02,13,1998,12,5.202216e+09,6220406.0,836.31,NaN,NaN,NaN
2,1998-0596-ZWE,596,Natural,Biological,Epidemic,Bacterial disease,Zimbabwe,ZWE,Eastern Africa,Africa,NaN,NaN,NaN,NaN,Vaccinated,12.0,NaN,335.0,NaN,335.0,NaN,NaN,NaN,NaN,NaN,NaN,1998-03-27,1998-01-22,64,1998,1,6.401968e+09,11747072.0,544.98,18.5,28.8,0.0
3,1999-0719-ZMB,719,Natural,Biological,Epidemic,Epidemic,Zambia,ZMB,Eastern Africa,Africa,NaN,NaN,NaN,NaN,Vaccinated,393.0,NaN,11327.0,NaN,11327.0,NaN,NaN,NaN,NaN,NaN,NaN,1999-06-15,1999-06-14,1,1999,6,3.404312e+09,10140561.0,335.71,NaN,NaN,NaN
4,2006-0727-PRY,727,Natural,Biological,Epidemic,Viral disease,Paraguay,PRY,South America,Americas,NaN,NaN,NaN,NaN,Vaccinated,17.0,NaN,100000.0,NaN,100000.0,NaN,NaN,NaN,NaN,NaN,NaN,2007-04-15,2006-11-22,144,2006,11,1.064616e+10,5913209.0,1800.40,17.4,33.0,NaN


**NOTE:** As we can see, Population, GDP and Weather data was added

In [18]:
df.to_csv('processed_data.csv', index = False)

In [19]:
df.columns

Index(['Dis No', 'Seq', 'Disaster Group', 'Disaster Subgroup', 'Disaster Type',
       'Disaster Subtype', 'Country', 'ISO', 'Region', 'Continent', 'Origin',
       'Associated Dis', 'AID Contribution ('000 US$)', 'Dis Mag Value',
       'Dis Mag Scale', 'Total Deaths', 'No Injured', 'No Affected',
       'No Homeless', 'Total Affected', 'Reconstruction Costs ('000 US$)',
       'Reconstruction Costs, Adjusted ('000 US$)',
       'Insured Damages ('000 US$)', 'Insured Damages, Adjusted ('000 US$)',
       'Total Damages ('000 US$)', 'Total Damages, Adjusted ('000 US$)',
       'End_Date', 'Start_Date', 'Duration', 'Year', 'Month', 'GDP',
       'Population', 'GDP_per_capita', 'tmin', 'tmax', 'prcp'],
      dtype='object')